In [271]:
import pandas as pd
import random
import scipy as sp
import numpy as np
import cvxpy as cvx
import mosek
import math

#### Load Data

In [272]:
dfTripData = pd.read_csv("location_data.csv")
dfLoadData=pd.read_csv("use_data_2wk.csv") #365 days, 1 min increment, 142 houses 
dfGenData=pd.read_csv("annual_capfac_15MinInterpolation.csv") #representative solar day, 15 min interval
dfAsPrices=pd.read_csv('20180701_20180701_PRC_AS_DAM_v1.csv')
dfCostElectric=pd.read_csv("TOU_Rate.csv") #15 min interal 4-9 pm, I think this is tou B

#### Script Parameters

In [273]:
intNumHouseNodes = 5
intNumCommNodes = 2
intTotalNodes=intNumHouseNodes+intNumCommNodes
intAvgNumCarsNode = 5 # goal is to have about 5 * 5 total cars
intSdNumCarsNode = 1
intProfiles = dfTripData.shape[0]

#not sure where these are used 
fltRegDownVal=4  
fltRegUpVal= 5   # they arent used in this range?

lsCostElectric =dfCostElectric.values.tolist()


# could make battery parameters a randomized instantiation
fltCarEff = 3.95 # mi/kWh
fltBatteryCap = 40. # kWh
fltChargeRate = 1.9 # kW
fltDt = 0.25 # delta T
fltHomeRate = 1.4 # kW
fltWorkRate = 7 # kW

#time and conversion intervals
int_minutes=int(15)
int_run_days=2 #day
int_run_hours=24 #hours
int_run_time_interval=60/int_minutes #
time_kwh=15/60  #convert kw data to kwh

#### Functions

In [274]:
# create residential nodes

def createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData):
    #initialize data frame to hold master
    dfNodesTrips = pd.DataFrame(columns=['TripId','ResNode','CommNode','HouseID'])  
    house=int(0)
    for intNode in range(intNumHouseNodes):

        # find how many vehicles will be at that residential node
        intNodeNumCars = int(random.gauss(intAvgNumCarsNode,intSdNumCarsNode))
        # assign cars to that node
        lsCarIds = [] # container to hold trip id for that node
        lsCommNode = [] # container to hold where the commerical node 
        lsResNode = [] # container to hold the res node
        lsVehicleHouses=[] #container to hold the house related to each vehicle/trip
        for intCar in range(intNodeNumCars):

            # find a car from the data frame
            ind = random.randint(0,intProfiles)
            lsCarIds.append(dfTripData['ID'].iloc[ind]) # add to car id container

            # save res node, added one because 1 initialized index was mismached with comm and load nodes
            lsResNode.append(intNode)  

            # find which commercial node that car goes to
            lsCommNode.append(random.randint(0,intNumCommNodes-1))
            
            #add a house id to associate with the trip id
            lsVehicleHouses.append(house)
            
            #iterate up house id, 0 based indexing to match the array calls/nodes
            house+=int(1)

        # create data frame and add to master
        dfNode = pd.DataFrame(list(zip(lsCarIds,lsResNode,lsCommNode,lsVehicleHouses)), \
                              columns=['TripId','ResNode','CommNode','HouseID'])

        dfNodesTrips = dfNodesTrips.append(dfNode)

    dfNodesTrips.reset_index(inplace=True, drop=True)
   
    intVeHouses=house
    
    return dfNodesTrips,intVeHouses
    

In [275]:

def convertLoadData(dfLoadData,dfNodesTrips,int_run_days,int_run_hours,int_minutes, intVeHouses,intNumCommNodes,intNumHouseNodes):
    '''
    process use data from 1 minute intervals to be on 15 minute interals and for specified run time, and associate with nodes and vehicles
    Each 24 hour time frame goes from 0 (midnight) to 24 (midnight)
    '''
    #initialize time frame to pull
    minutes=60*int_run_hours
    run_length=minutes*int_run_days
    intIntervals=int(run_length/int_minutes)
    
    #clean dataframe and convert to numpty for manipulation
    dfLoadData = dfLoadData.dropna(axis=1,thresh=100) #pull only nonzeros columns, but all of them have like one so just set a threshhold
    npLoadData=dfLoadData.to_numpy()
    run_loads=npLoadData[0:run_length,2:]    # run lenth extracted from dataset, 130+ loads included, skip timestep/day
    run_loads=run_loads.astype("float32")#I think iteration is faster in an np array? 
    #Initialize profile length to pull 
    intNumLoads = intVeHouses
    nLoads=len(run_loads[0,:]) 
   
    #container for house loads
    arrHouseLoads=np.zeros((intNumLoads, intIntervals))  #loadsxtime
    #process to 15 minutes fgor houses
    for n in range(intNumLoads):  
        random_select=random.randrange(0,nLoads,1)
        house_load_1min=run_loads[:,random_select] #1 house one vehicle
        o=int(0)
        j=0
        while o < (run_length-int_minutes): 
            start=o
            end=o+int_minutes
            new_interval=np.nanmean(house_load_1min[start:end])
            arrHouseLoads[n,j]=new_interval
            o+=int_minutes
            j+=1
            
    
    #calculate cumulative home nodes 
    dfNodeLoads=pd.DataFrame()  #rows will be t, columns will be node ID information, reverse of other df for easy lookup by node
    dctHomeNode=dict() #save node to id mapping, may be useful
    cumulativeLoads=np.zeros((intIntervals))
    i=0 #columns in the node dataframe
    
    for n in range(intNumHouseNodes):  
        #find all the houses that match that Node
        NodeID='ResNode'+str(n)
        lsHomeNode=dfNodesTrips[dfNodesTrips['ResNode']==int(n)].index.tolist()#all home/vehicles that have that resnode associated in dfNodesTrips
        sumrows=arrHouseLoads[lsHomeNode,:]
        cumulativeLoads=np.nansum(sumrows,axis=0)  #this should just be one column
        dctHomeNode[NodeID]=lsHomeNode
        dfNodeLoads.insert(i,NodeID,cumulativeLoads,True)
        i+=1
    
    arrCommercialLoads=np.zeros((intIntervals,intNumCommNodes))
    #commercial node loads, related to multiple vehicles 
    for k in range(intNumCommNodes):  #home x10 for commercial, should replace with real commercial values if possible! 
        random_select=random.randrange(0,nLoads,1)
        comm_load_1min=50*run_loads[:,random_select]
        o=int(0)
        j=0
        while o < (run_length-int_minutes): 
            start=o
            end=o+int_minutes
            new_interval=np.nanmean(comm_load_1min[start:end])
            arrCommercialLoads[j,k]=new_interval
            o+=int_minutes
            j+=1
        dfNodeLoads.insert(i,"CommNode"+str(k),arrCommercialLoads[:,k],True) 
        i+=1
    
    PeakTimes=dfNodeLoads.idxmax()
    PeakLoads=dfNodeLoads.max()

    
    lsNodes=dfNodeLoads.columns.values.tolist()
    return   arrHouseLoads,dfNodeLoads, PeakTimes,PeakLoads, lsNodes, dctHomeNode

In [276]:
def SolarGenData(dfGenData,int_run_days,int_run_hours,int_minutes,intVeHouses,intNumCommNodes,intNumHouseNodes, dctHomeNode):
    '''
    Process solar data to different solar capacities for each house (no commercial solar)
    '''
    #gendata is capfac at 15 minute intervals for 24 hours 
    run_schedule=int((60/int_minutes)*int_run_hours) #intervals
    intIntervals=int(run_schedule*int_run_days)
    intNumHomeSolar=intVeHouses
    #extract solar data
    npGenData=dfGenData.to_numpy() #I think iteration is faster in an np array?
    npGenData=npGenData[:,2]
    
    #initial house solar gen container 
    arrHouseGen=np.zeros((intNumHomeSolar,intIntervals))
    #every household must average 1 kw generation
    
    house_sol_scale=1 #kw
    comm_sol_scale=10 #kw
    #total solar gen for each house associated with a vehicle
    for n in range(intNumHomeSolar):
        for d in range(int_run_days):
            start=0
            end=start+run_schedule
            arrHouseGen[n,start:end]=np.multiply(npGenData[:],house_sol_scale)
    
    arrCommGen=np.zeros((intIntervals,intNumCommNodes))
    #node level generation
    dfNodeGens=pd.DataFrame()
    i=0
    for n in range(intNumHouseNodes):  
        NodeID='ResNode'+str(n)
        lsHomeNode=dctHomeNode[NodeID]
        sumrows=arrHouseGen[lsHomeNode,:]
    
        cumulativesolar=np.nansum(sumrows,axis=0)
        
        dfNodeGens.insert(i,NodeID,cumulativesolar,True)
        i+=1 
    for k in range(intNumCommNodes):  #home x10 for commercial, should replace with real commercial values! 
        for d in range(int_run_days):
            start=0
            end=start+run_schedule
            arrCommGen[start:end,k]=npGenData[:]*comm_sol_scale #5 house sum per node
        dfNodeGens.insert(i,"CommNode"+str(k),arrCommGen[:,k],True)
    i+=1
        
    return arrHouseGen,dfNodeGens,npGenData

In [277]:
def MapCarNodesTime(int_minutes,int_run_hours,lsVehicleHouses,dfNodesTrips,dfTripData):
    #make an array that has x vehicles by 96, with location ID, sum of total for each loc
    '''
    Function to find the maximum charging rate at each time interval and whether it is
    available to charge, and the node at which it is present 
    '''
   
    intervals=int((60/int_minutes)*int_run_hours) #intervals for one run
    intIntervals=int(intervals*int_run_days)
    intNumProfs=intVeHouses
    # this will hold the charging rate in kW
    arrChargeRate = np.zeros((intNumProfs, intervals)) 
    # this will hold the binary of whether the vehicle can charge
    arrCanCharge = np.zeros((intNumProfs, intervals))
    arrChargeLoc = np.empty((intNumProfs, intervals),dtype=object)

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # get location sequence for that trip
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].iloc[0,3:-1].values

        # iterate throught the locations to get its charge rate and availability 
        for intInterval, strLoc in enumerate(lsLocations):
            if strLoc == 'H': # if at home max charge 
                arrChargeRate[ind, intInterval] = fltHomeRate
                arrCanCharge[ind,intInterval] = 1
                arrChargeLoc[ind,intInterval]='ResNode'+str(dfNodesTrips.iloc[ind].loc['ResNode'])
            elif strLoc == 'W': # working charge rate
                arrChargeRate[ind, intInterval] = fltWorkRate
                arrChargeRate[ind,intInterval] = 1
                arrChargeLoc[ind,intInterval]='CommNode'+str(dfNodesTrips.iloc[ind].loc['CommNode'])
            else: # you are moving and can't charge
                arrChargeRate[ind, intInterval] = 0
                arrChargeRate[ind, intInterval] = 0
                arrChargeLoc[ind,intInterval]="Away"
    
    x=arrChargeRate
    y=arrCanCharge
    z=arrChargeLoc
    for n in range(1,int_run_days):
        arrChargeRate=np.append(arrChargeRate,x,axis=1)
        arrCanCharge=np.append(arrCanCharge,y,axis=1)
        arrChargeLoc=np.append(arrChargeLoc,z,axis=1)
                   

    return arrChargeRate, arrCanCharge, arrChargeLoc
    

In [278]:
def netLoadChargeLoc(arrHouseGen,arrHouseLoads,arrChargeLoc,dfNodeLoads,dfNodeGens,intVeHouses,intTotalNodes,lsNodes,int_minutes,int_run_hours):
    '''
    Find the net kw at each charge pt locations
    '''
    intNumProfs=intVeHouses
    intervals=int(60/int_minutes*int_run_hours)
    intIntervals=int(intervals*int_run_days)
    arrNetNodeLoad=np.zeros((intTotalNodes,intIntervals))
    arrNetHomeLoad=np.zeros((intNumProfs,intIntervals))
    dfNetNodeLoad=pd.DataFrame()
    #use charge location as values to update to load value, ie charge location=column name, 
  
    #by house/vehicle net load & gen arrays now match for homes    
    arrNetHomeLoad=np.subtract(arrHouseLoads,arrHouseGen)
    
    #total net load per node; aggregated house gen/solar gen and commercial load/solar gen 
    dfNetNodeLoad=dfNodeLoads.subtract(dfNodeGens)
    
    NameDict={}
    #had issue saving a flattened array to the 
    dfNodeIdentities=pd.DataFrame()
    
    
    #create naming dict 
    for n in lsNodes:
        NameDict[n]="arrID"+str(n)
    
    #create identity arrays for the charge locations for each node at each timestep
    i=0
    for n in lsNodes:
        name=str(NameDict[n])
        NameDict[n]=np.zeros((intNumProfs,intIntervals))
        for i in range(intIntervals):
            lsLoc=np.where(arrChargeLoc[:,i]==n)
            NameDict[n][lsLoc,i]=1
        FlatArray=NameDict[n].flatten()
        dfNodeIdentities[name]=FlatArray 
        i+=1 
        
    return dfNetNodeLoad,arrNetHomeLoad,dfNodeIdentities

In [279]:
def findVehicleConsumption(dfNodesTrips, fltCarEff, fltDt):

    '''
    Function to find the power consumption of the vehicle at each time step
    from driving
    '''

    intNumProfs = dfNodesTrips.shape[0]
    arrConsumption = np.zeros((intNumProfs, 96))

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # find miles travelled
        fltMiles = dfTripData.loc[dfTripData['ID'] == strId,'DIST_M'].values[0]

        # get a list of locations
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].values.flatten()[3:-1].tolist()

        # so that we can find how many intervals we are away
        intAway = sum(1 if strLoc == "A" else 0 for strLoc in lsLocations)

        # miles per interval of travel and kwh per interval
        fltMiPerInt = fltMiles / intAway
        fltEnergyInt = fltMiPerInt / fltCarEff / fltDt # miles / (miles/kWh) / 0.25 hrs

        lsCarEnergy = [fltEnergyInt*-1 if strLoc == 'A' else 0 for strLoc in lsLocations ] 

        arrConsumption[ind,:] = lsCarEnergy
    
    return arrConsumption
    

In [280]:
def getSOCconstraints(dfNodesTrips, fltCarEff, fltChargeRate, fltDt):

    intNumProfs = dfNodesTrips.shape[0]
    arrSOCrequirements = np.zeros((intNumProfs, 96))
##this looks like its only treating each car like it has a single travel period, rather than multiple?
    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # find miles travelled
        fltMiles = dfTripData.loc[dfTripData['ID'] == strId,'DIST_M'].values[0]

        # get a list of locations
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].values.flatten()[3:-1].tolist()

        # so that we can find how many intervals we are away
        intAway = sum(1 if strLoc == "A" else 0 for strLoc in lsLocations)

        # and then find the average miles travelled each interval
        fltAvgMileInterval = fltMiles / intAway

        intStart = 0 
        while intStart < 96 and 'A' in lsLocations[intStart:]:

            # Find the first first time we start moving
            intStart = intStart + lsLocations[intStart:].index('A') 

            # from there find when we arrive at a destination
            if 'W' in lsLocations[intStart:] and 'H' in lsLocations[intStart:]:
                intEnd = min(lsLocations[intStart:].index('H'), lsLocations[intStart:].index('W'))
            elif 'W' in lsLocations[intStart:]:
                intEnd = lsLocations[intStart:].index('W')
            elif 'H' in lsLocations[intStart:]:
                intEnd = lsLocations[intStart:].index('H')
            else: # we are going to assume they are driving the rest of the time??
                break
               

            # intervals driving for trip is then (intStart+intEnd +1) - intStart = intEnd =1
            # then we can find miles travelled for first trip 
            fltMilesTraveled = fltAvgMileInterval * (intEnd)
            fltKwhRequirement = fltMilesTraveled / fltCarEff # miles / (miles/kWh)

            # now find SOC required
            fltInitialSoc = fltKwhRequirement

            # so we need to to have the initial SOC at the interval before this time:
            arrSOCrequirements[ind,intStart-1] = fltInitialSoc
            
            # now let's back fill in SOC from there
            intNext = 2
            fltPreviousSOC = fltInitialSoc - fltChargeRate * fltDt
            while fltPreviousSOC > 0:
                
                # fill in the SOC needed in the previous time series
                arrSOCrequirements[ind, intStart - intNext] = fltPreviousSOC
                
                intNext += 1 # for next iteration go to the next previous time step
                fltPreviousSOC = fltPreviousSOC - fltChargeRate * fltDt
           

            intStart += intEnd # increment the starting point for finding the next trip

    return arrSOCrequirements


In [281]:
def getChargeRateConstraints(dfNodesTrips, dfTripData):
    '''
    Function to find the maximum charging rate at each time interval and whether it is
    available to charge. 
    '''

    intNumProfs = dfNodesTrips.shape[0]
    # this will hold the charging rate in kW
    arrChargeRate = np.zeros((intNumProfs, 96)) 
    # this will hold the binary of whether the vehicle can charge
    arrCanCharge = np.zeros((intNumProfs, 96))
    

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # get location sequence for that trip
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].iloc[0,3:-1].values

        # iterate throught the locations to get its charge rate and availability 
        for intInterval, strLoc in enumerate(lsLocations):

            if strLoc == 'H': # if at home max charge 
                arrChargeRate[ind, intInterval] = fltHomeRate
                arrCanCharge[ind,intInterval] = 1
            elif strLoc == 'W': # working charge rate
                arrChargeRate[ind, intInterval] = fltWorkRate
                arrChargeRate[ind,intInterval] = 1
            else: # you are moving and can't charge
                arrChargeRate[ind, intInterval] = 0
                arrChargeRate[ind, intInterval] = 0
    
    return arrChargeRate, arrCanCharge

In [282]:
def getInitialSoc(arrSOCconstraint): 
    '''
    Function to get an random initialization of the vehicle's SOC
    '''
    
    intNumVehicles = np.shape(arrSOCconstraint)[0]
    lsInitialSoc = np.zeros((intNumVehicles)) # holder to have the initial SOC of vehicles

    for intVehicle in range(intNumVehicles): # for each vehicle 

        fltSocStart = random.random() * fltBatteryCap
        
        # make sure that the random SOC is > the minimum SOC required
        while fltSocStart < arrSOCconstraint[intVehicle,0]:
            fltSocStart = random.random() * fltBatteryCap
        
        lsInitialSoc[intVehicle] = fltSocStart
            
    arrSOCconstraint[:,0] = lsInitialSoc
    
    return arrSOCconstraint

In [283]:
def getAsValues(dfAsPrices):
    dfAsPricesRu = dfAsPrices.loc[(dfAsPrices['ANC_REGION'] == 'AS_CAISO_EXP') \
                                    & (dfAsPrices['ANC_TYPE'] == 'RU')].sort_values('OPR_HR').reset_index()
    dfAsPricesRd = dfAsPrices.loc[(dfAsPrices['ANC_REGION'] == 'AS_CAISO_EXP') & \
                                  (dfAsPrices['ANC_TYPE'] == 'RD')].sort_values('OPR_HR').reset_index()

    lsRuValue = []
    lsRdValue = []

    for intHr in range(24):

        # get hourly $/MW price -- convert to 15 minute kWh
        fltUp = dfAsPricesRu['MW'].iloc[intHr]*fltDt/1000 
        fltDown = dfAsPricesRd['MW'].iloc[intHr]*fltDt/1000

        for intInterval in range(4):
            lsRuValue.append(fltUp)
            lsRdValue.append(fltDown)
    
    return lsRuValue, lsRdValue

#### Execute

In [284]:
def BatteryDegradationModel ():
    # dummy values -- MUST UPDATE
    #velvet->debugged, but wasnt sure intent on some of these, should this actually be part of a function that is called in the objective function?

    # battery constants
    fltAlphaSei = 5.75 * 10**-2
    fltBetaSei = 121
    fltKdelta1 = 1.40 * 10**5
    fltKdelta2 = -5.01 * 10**-1
    fltKdelta3= 1.23*10**5
    fltKsigma = 1.04
    fltSigmaRef = 0.5
    fltKtime = 4.14* 10**-10 # 1/s
    fltKtemp = 6.93 * 10**-2
    fltTref = 25 
    fltCost = 7000 # cost of Tesla battery replacement

    fltTemp=25
    fltDepthofDischarge=0.6
    fltStateofCharge=0.7
    fltTime=3600*int_run_hours # time in seconds


    fltStressTemp=math.exp(fltKtemp*(fltTemp-fltTref))*fltTref/fltTemp
    fltStressSOC=math.exp(fltKsigma*(fltStateofCharge-fltSigmaRef))
    fltStressTime=fltKtime*fltDt  #not clear what t is supposed to be 
    fltStressDischarge=(fltKdelta1*fltDepthofDischarge**fltKdelta2+fltKdelta3)**(-1)

    fltDegradation=(fltStressDischarge+fltStressTime)*fltStressSOC*fltStressTemp

    #varBatteryLife=1-fltAlphaSei*math.exp((-varNumberOfCycles*fltBetaSei*fltDegradation))-(1-fltAlphaSei)*math.exp((-varNumberOfCycles*fltDegradation))
    #need to double check the inputs 
        
    return fltAlphaSei,fltBetaSei,fltDegradation

In [285]:
# call functions to get Nodes Dataframe
dfNodesTrips, intVeHouses = createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData)

#create df of loads for commercial & residential nodes
arrHouseLoads,dfNodeLoads, PeakTimes,PeakLoads, lsNodes,dctHomeNode=convertLoadData(dfLoadData,dfNodesTrips, \
                                    int_run_days,int_run_hours,int_minutes, intVeHouses,intNumCommNodes,intNumHouseNodes)

#create generation for 15 min increments for each node
arrHouseGen,dfNodeGens,npGenData=SolarGenData(dfGenData,int_run_days,int_run_hours,int_minutes, \
                                              intVeHouses,intNumCommNodes,intNumHouseNodes, dctHomeNode)


# now call function to get the SOC constraints array
arrSOCconstraint = getSOCconstraints(dfNodesTrips, fltCarEff, fltChargeRate, fltDt)

# call function to get an array of maximum charging rates and map to location by node
arrChargeRate, arrCanCharge, arrChargeLoc=MapCarNodesTime(int_minutes,int_run_hours,intVeHouses,dfNodesTrips,dfTripData)


# call function to get initial SOCs of vehicles
# replaces first column of soc constraint array
arrSOCconstraint = getInitialSoc(arrSOCconstraint)  #should this have a different name?

# call function to get power consumption of the vehicle
arrConsumption = findVehicleConsumption(dfNodesTrips, fltCarEff, fltDt)

#Get Ancillarys service cost values 
lsRuValue, lsRdValue=getAsValues(dfAsPrices)

dfNetNodeLoad,arrNetHomeLoad,dfNodeIdentities=netLoadChargeLoc(arrHouseGen,arrHouseLoads,arrChargeLoc,dfNodeLoads,dfNodeGens,intVeHouses,\
                                              intTotalNodes,lsNodes,int_minutes,int_run_hours)

fltAlphaSei,fltBetaSei,fltDegradation=BatteryDegradationModel()



ResNode0       9.692067
ResNode1      15.997067
ResNode2      12.752267
ResNode3      10.296733
ResNode4      18.638423
CommNode0    346.309998
CommNode1     60.936665
dtype: float64


#### Define OptimizationFunctions

In [323]:
def make_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dfNodeIdentities,lsNodes, \
                     fltDt, fltBatteryCap, arrConsumption,PeakLoads,intTotalNodes):
    
#I thought you couldn't use np functions inside cvx? may want to pass these in as variables instead
    intVehicles = np.shape(arrSOCconstraint)[0]
    intTimes = np.shape(arrSOCconstraint)[1]
    
    # make variables
    varCharge = cvx.Variable((intVehicles,intTimes)) # charging rate of vehicles
    varDischarge = cvx.Variable((intVehicles,intTimes)) # discharging rate of vehicles
    varSOCs = cvx.Variable((intVehicles,intTimes)) # SOCs of the vehicles
    varBatteryLife=cvx.Variable((intVehicles)) #batter life of the vehicle remaining
    varNumberOfCycles=cvx.Variable(intVehicles)
    varDegradationCost=cvx.Variable((intVehicles))
    varNodeSum=cvx.Variable((intTotalNodes,intTimes))
    # initialzie constraints
    constraints = []
    
    # define the charge rate rate constraints
    constraints += [varCharge >= 0] # positively defined
    constraints += [varDischarge <= 0] # negatively defined
    constraints += [varSOCs >= 0] # can't have negative SOC
    constraints += [varSOCs <= fltBatteryCap] # can't over charge
    constraints += [varDegradationCost>=0] #positively constrained 
    constraints += [varNodeSum>=0]
    constraints += [varNumberOfCycles>=0]
    
    
    #create node summation values for each vehicle at each time step to apply node level. I think this may need to be an intermediate variable that sums the vehicles together for the node?
    #Try out the identity matrix for each node*charge[v,t] array<transformer max[t] for each node!!
    
    for v in range(intVehicles):
 # 
        # loop through each time to model charging and SOC constraints
        for t in range(intTimes):
            
            # can only charge if present and max rate is given by array
            constraints += [varCharge[v, t] <= arrChargeRate[v,t]*arrCanCharge[v, t]]
            constraints += [-1*arrChargeRate[v,t]*arrCanCharge[v, t] <= varDischarge[v,t]]
            
            # vehicle's SOC must be greater than the minimum requirement
            constraints += [varSOCs[v,t] >= arrSOCconstraint[v,t]]
            
        
        for t in range(1,intTimes): # loop through from time 1 to end to set SOC
            
            constraints += [varSOCs[v,t]  == varSOCs[v,t-1] + varCharge[v,t]*fltDt + varDischarge[v,t]*fltDt + arrConsumption[v,t]*fltDt]
            
   #I think i need to extract the identity arrays/save them to a dynamic variable that can expand to hold as many nodes as we may end up having?? Reshape is not working below
   # n=0
   # for node in lsNodes:
   #     name='arrID'+node
   #     TransformerMax=PeakLoads[node]*1.2  #max is 20% above max load
   #     arrNodeIdentities=dfNodeIdentities[name].to_numpy()
     #reshape doesn't work, can just use the old fashion iteration or need to create dynamic variable as above
   #     nodeloc=np.reshape(np.ravel(arrNodeIdentities),(intVehicles,intTimes))
   #     for t in range (intTimes):
   #         constraints+= [varNodeSum[n,t] == cvx.sum(nodeloc[:,t]*varCharge[:,t])]
   #         constraints+= [varNodeSum[n,t] <= TransfromerMax]
   #     n+=1    
    
    return constraints, varCharge, varDischarge, varSOCs, varNumberOfCycles, varDegradationCost
       
    

In [332]:
def make_objectives(constraints, varCharge, varDischarge, varSOCs, varNumberOfCycles, varDegradationCost,\
                    arrCanCharge, arrChargeRate,arrSOCconstraint,fltBatteryCap,fltDt, fltRegDownVal,\
                    fltRegUpVal,lsCostElectric,lsRdValue,lsRuValue,fltAlphaSei,fltBetaSei,fltDegradation):
    
    
    intVehicles = np.shape(arrSOCconstraint)[0]
    intTimes = np.shape(arrSOCconstraint)[1]
    # define regulation up and down variables
    varRegUp = cvx.Variable((np.shape(arrCanCharge))) #this array is # vehiclesxtimesteps
    varRegDown = cvx.Variable((np.shape(arrCanCharge)))
    varNumberOfCycles=cvx.Variable(intVehicles)
    varDegradationCost=cvx.Variable(intVehicles)

    # always positive
    constraints += [varRegUp >= 0]
    constraints += [varRegDown >= 0]
    constraints += [varDegradationCost>=0] #positively constrained 
   
    for v in range(intVehicles):
        constraints+= [varNumberOfCycles[v] == (cvx.sum(varCharge, axis=1) + \
                                     cvx.sum(cvx.abs(varDischarge),axis=1))*fltDt/fltBatteryCap]
        constraints+= [varDegradationCost[v] == 4.66*varNumberOfCycles[v]]
            
        for t in range(intTimes):
    
        #   reg down constraints
            # it is at most the charging rate if it's available
            constraints += [varRegDown[v,t] <= (arrCanCharge[v,t]*arrChargeRate[v,t] - varCharge[v,t])*lsRdValue[t]]
            # or what is left to charge batteries capacity
            constraints += [varRegDown[v,t] <= ((fltBatteryCap - varSOCs[v,t])/fltDt - varCharge[v,t])*lsRdValue[t]]
            
            # reg up constraints 
            # it is at most the charge rate if it's avalable 
            constraints += [varRegUp[v,t] <= (arrCanCharge[v,t]*arrChargeRate[v,t] + varDischarge[v,t])*lsRuValue[t]]
            # or what it's minimum SOC requriement is
            constraints += [varRegUp[v,t] <= ((varSOCs[v,t] - arrSOCconstraint[v,t])/fltDt + varDischarge[v,t])*lsRuValue[t]]
        
    obj_value = cvx.sum(varRegDown) + \
                cvx.sum(varRegUp) - \
                cvx.sum(lsCostElectric*cvx.sum(varCharge,axis=0)) - \
               cvx.sum(varDegradationCost)
    
    return constraints, obj_value, varRegUp, varRegDown

In [333]:
constraints, varCharge, varDischarge, varSOCs, varNumberOfCycles, varDegradationCost = make_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dfNodeIdentities,lsNodes, \
                     fltDt, fltBatteryCap, arrConsumption,PeakLoads,intTotalNodes)
constraints, obj_value, varRegUp, varRegDown =  make_objectives(constraints, varCharge, varDischarge, varSOCs, varNumberOfCycles, varDegradationCost,\
                    arrCanCharge, arrChargeRate,arrSOCconstraint,fltBatteryCap,fltDt, fltRegDownVal,\
                    fltRegUpVal,lsCostElectric,lsRdValue,lsRuValue,fltAlphaSei,fltBetaSei,fltDegradation)
    
# convex usually needs to be minimized
obj = cvx.Maximize(obj_value)
prob = cvx.Problem(obj, constraints)

prob.solve(solver=cvx.MOSEK)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
var1181095[0] == (Sum(var1122208, 1, False) + Sum(abs(var1122209), 1, False)) * Promote(0.25, (22,)) / Promote(40.0, (22,)) , because the following subexpressions are not:
|--  var1181095[0] == (Sum(var1122208, 1, False) + Sum(abs(var1122209), 1, False)) * Promote(0.25, (22,)) / Promote(40.0, (22,))
var1181095[1] == (Sum(var1122208, 1, False) + Sum(abs(var1122209), 1, False)) * Promote(0.25, (22,)) / Promote(40.0, (22,)) , because the following subexpressions are not:
|--  var1181095[1] == (Sum(var1122208, 1, False) + Sum(abs(var1122209), 1, False)) * Promote(0.25, (22,)) / Promote(40.0, (22,))
var1181095[2] == (Sum(var1122208, 1, False) + Sum(abs(var1122209), 1, False)) * Promote(0.25, (22,)) / Promote(40.0, (22,)) , because the following subexpressions are not:
|--  var1181095[2] == (Sum(var1122208, 1, False) + Sum(abs(var1122209), 1, False)) * Promote(0.25, (22,)) / Promote(40.0, (22,))

In [ ]:
np.max(varRegDown.value)

In [ ]:
np.max(varRegUp.value)